In [1]:
# Multilayer Perceptron (MLP) for multi-class softmax classification:
# modified from 
# https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam

import random

C:\Users\ortci\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

n_samples = 1000000
n_partitions = 10

# Generate tasks and partitions
partition_data = np.random.random((n_samples, n_partitions)) # partition data generation

data = np.zeros((n_samples, n_partitions))                   # initialize input layer
labels = np.zeros((n_samples, n_partitions))                 # initialize outputs layer for training 

task_data = np.zeros((n_samples, 1))                         # initialize task list

# fails to account a 'perfect' fit where fit == 0
for i in range (0, n_samples):
    
    partitions = partition_data[i]
    task = random.uniform(0, partitions.max())
    task_data[i] = task
    
    best_partition = -1
    best_fit = 999999999
    
    for j in range (0, n_partitions):
        current_fit = partitions[j] - task
        data[i,j] = current_fit
        if current_fit > 0 and current_fit < best_fit:
            best_fit = current_fit
            best_partition = j
    
    labels[i][best_partition] = 1
    

X = np.hstack((task_data,partition_data))
y = labels

In [3]:
# split data between train and test set 
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [4]:
import tensorflow as tf
from keras.models import load_model

model = load_model('MLP_elu_softmax_11_inputs.h5')

# Freeze all layers in first model
for layer in model.layers:
    layer.trainable = False

# Construct 2nd half model
n_nodes = 32
model.add(Dense(n_nodes, activation='relu', name='layerA'))
model.add(Dense(n_nodes, activation='relu', name='layerB'))
model.add(Dense(n_partitions, activation='softmax', name='softmax'))

model.load_weights('simple_classifer1_wts.h5', by_name=True)

# optimizer options
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = tf.train.RMSPropOptimizer(0.008)
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [5]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.models import load_model
import pydot
import graphviz
from keras.utils import plot_model
plot_model(model, to_file='combined_model_config.png', show_shapes=True)

In [6]:
model.compile(optimizer=tf.train.RMSPropOptimizer(0.002),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

In [7]:
batchsize = 256

model.fit(X_train, y_train,
          epochs=10,
          batch_size=batchsize)
score = model.evaluate(X_test, y_test, batch_size=batchsize)
print(score)

Epoch 1/10
800000/800000 [==============================] - 16s 19us/step - loss: 1.7955 - categorical_accuracy: 0.4182
Epoch 2/10
800000/800000 [==============================] - 16s 20us/step - loss: 1.3413 - categorical_accuracy: 0.4639
Epoch 3/10
800000/800000 [==============================] - 17s 21us/step - loss: 1.2896 - categorical_accuracy: 0.4633
Epoch 4/10
800000/800000 [==============================] - 17s 21us/step - loss: 1.2689 - categorical_accuracy: 0.4643
Epoch 5/10
800000/800000 [==============================] - 17s 21us/step - loss: 1.2551 - categorical_accuracy: 0.4658
Epoch 6/10
800000/800000 [==============================] - 16s 20us/step - loss: 1.2390 - categorical_accuracy: 0.4709
Epoch 7/10
800000/800000 [==============================] - 16s 20us/step - loss: 1.2381 - categorical_accuracy: 0.4729
Epoch 8/10
800000/800000 [==============================] - 16s 20us/step - loss: 1.2362 - categorical_accuracy: 0.4732
Epoch 9/10
800000/800000 [==============

In [8]:
'''
Why is the training loss much higher than the testing loss?

A Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and 
L1/L2 weight regularization, are turned off at testing time. Besides, the training loss is the average 
of the losses over each batch of training data. Because your model is changing over time, the loss over 
the first batches of an epoch is generally higher than over the last batches. On the other hand, the 
testing loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.
'''

'\nWhy is the training loss much higher than the testing loss?\n\nA Keras model has two modes: training and testing. Regularization mechanisms, such as Dropout and \nL1/L2 weight regularization, are turned off at testing time. Besides, the training loss is the average \nof the losses over each batch of training data. Because your model is changing over time, the loss over \nthe first batches of an epoch is generally higher than over the last batches. On the other hand, the \ntesting loss for an epoch is computed using the model as it is at the end of the epoch, resulting in a lower loss.\n'

In [9]:
from keras.models import load_model

model.save('combined3.h5')  # creates a HDF5 file 'my_model.h5'
model.save_weights('combined3_weights.h5')

'''
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('MLP_Multiclass_softmax_10_inputs.h5')
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture
model.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture
'''

C:\Users\ortci\Anaconda3\lib\site-packages\keras\engine\saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


"\ndel model  # deletes the existing model\n\n# returns a compiled model\n# identical to the previous one\nmodel = load_model('MLP_Multiclass_softmax_10_inputs.h5')\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5') # for same architecture\nmodel.load_weights('MLP_Multiclass_softmax_10_inputs_weights.h5', by_name=True) # for different architecture\n"

array([[0.00395757, 0.18941276, 0.34643471, ..., 0.33949192, 0.71779765,
        0.99871352],
       [0.78894385, 0.56357662, 0.88965427, ..., 0.17649624, 0.46990644,
        0.58300665],
       [0.5222747 , 0.42758563, 0.58371769, ..., 0.9484488 , 0.06577278,
        0.73940218],
       ...,
       [0.55871245, 0.24221679, 0.55802383, ..., 0.74247317, 0.65693693,
        0.09944731],
       [0.76269568, 0.01646626, 0.69072538, ..., 0.04114213, 0.89672438,
        0.80557408],
       [0.50144768, 0.3803096 , 0.75814613, ..., 0.83841984, 0.95195423,
        0.7731539 ]])